# Sentiment Analysis


In [18]:
%matplotlib notebook
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from wordcloud import WordCloud as wc
from nltk.corpus import stopwords
import matplotlib.pylab as pylab
import matplotlib.pyplot as plt
from pandas import get_dummies
import matplotlib as mpl
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib
import warnings
import sklearn
import string
import scipy
import numpy
import nltk
import json
import sys
import csv
import os

In [19]:
print('matplotlib: {}'.format(matplotlib.__version__))
print('sklearn: {}'.format(sklearn.__version__))
print('scipy: {}'.format(scipy.__version__))
print('seaborn: {}'.format(sns.__version__))
print('pandas: {}'.format(pd.__version__))
print('numpy: {}'.format(np.__version__))
print('Python: {}'.format(sys.version))


matplotlib: 3.0.2
sklearn: 0.20.1
scipy: 1.1.0
seaborn: 0.9.0
pandas: 0.23.4
numpy: 1.15.4
Python: 3.7.1 (default, Dec 10 2018, 22:54:23) [MSC v.1915 64 bit (AMD64)]


In [20]:
pd.set_option('display.max_colwidth', 100)


## Problem Statement

In [21]:
business = pd.read_csv("C:\\Users\Aravind\Documents\Datasets\Yelp review\yelp_business.csv")

In [22]:
business.head()

,business_id,name,neighborhood,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories
0,FYWN1wneV18bWNgQjJ2GNg,"""Dental by Design""",NaN,"""4855 E Warner Rd, Ste B9""",Ahwatukee,AZ,85044,33.330690,-111.978599,4.0,22,1,Dentists;General Dentistry;Health & Medical;Oral Surgeons;Cosmetic Dentists;Orthodontists
1,He-G7vWjzVUysIKrfNbPUQ,"""Stephen Szabo Salon""",NaN,"""3101 Washington Rd""",McMurray,PA,15317,40.291685,-80.104900,3.0,11,1,Hair Stylists;Hair Salons;Men's Hair Salons;Blow Dry/Out Services;Hair Extensions;Beauty & Spas
2,KQPW8lFf1y5BT2MxiSZ3QA,"""Western Motor Vehicle""",NaN,"""6025 N 27th Ave, Ste 1""",Phoenix,AZ,85017,33.524903,-112.115310,1.5,18,1,Departments of Motor Vehicles;Public Services & Government
3,8DShNS-LuFqpEWIp0HxijA,"""Sports Authority""",NaN,"""5000 Arizona Mills Cr, Ste 435""",Tempe,AZ,85282,33.383147,-111.964725,3.0,9,0,Sporting Goods;Shopping
4,PfOCPjBrlQAnz__NXj9h_w,"""Brick House Tavern + Tap""",NaN,"""581 Howe Ave""",Cuyahoga Falls,OH,44221,41.119535,-81.475690,3.5,116,1,American (New);Nightlife;Bars;Sandwiches;American (Traditional);Burgers;Restaurants


In [23]:
business[business['name'].str.contains("Taco Guild")]
#https://www.yelp.com/biz/taco-guild-phoenix-2?osq=Brick+House+Tavern

,business_id,name,neighborhood,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories
7834,3C5Z9homtzkWHouH2BHXYQ,"""Taco Guild""",NaN,"""546 E Osborn Rd""",Phoenix,AZ,85012,33.487747,-112.065665,4.0,1092,1,Mexican;Bars;Nightlife;Restaurants;Cocktail Bars;Tex-Mex;Gastropubs


In [24]:
reviews = pd.read_csv("C:\\Users\\Aravind\\Documents\\Datasets\\Yelp review\\reviews_sample_5k.csv")


In [25]:
reviews[reviews['business_id'] == '3C5Z9homtzkWHouH2BHXYQ']

,Unnamed: 0,review_id,user_id,business_id,stars,date,text,useful,funny,cool
510,3832693,SOz6z3PHKKbGeviori0KKQ,PTE6qLNpRtxrJFsZalV-Gg,3C5Z9homtzkWHouH2BHXYQ,1,2017-04-02,Tacos were very bland. Queso tasted like it was from a can. Female bartender never once smiled. ...,0,0,0
2721,700793,VF21xhGJYi3YuEOFBPc11A,-XXNYOT4fiuH2-TbqfLVLQ,3C5Z9homtzkWHouH2BHXYQ,4,2017-01-23,This place is perfect if you are trying to impress someone on a date. It has a great vibe and ...,0,0,0
4807,2918551,_-rpTzAbsgTuVsiVBA7mgQ,EFi0P4NywAhZDLoFKIJPFQ,3C5Z9homtzkWHouH2BHXYQ,5,2014-05-16,"really liked the tacos - we had duck, lamb and a few others...and enjoyed the sangria. the build...",0,0,0


In [26]:
pd.set_option('display.max_colwidth', -1)
# Bad Review 
reviews[reviews['review_id'] == 'VF21xhGJYi3YuEOFBPc11A']['text']

2721    This place is perfect if you are trying to impress someone on a date.   It has a great vibe and decor.  I would recommend it for drinks and appetizers.   The dishes here are good,  but take a different approach to flavor than the traditional Mexican cuisine.  For example the green enchiladas here don't come with a traditional tomatillo sauce.  Great for a date,  but don't expect traditional flavors.
Name: text, dtype: object

In [27]:
# 5 star review
reviews[reviews['business_id'] == '3C5Z9homtzkWHouH2BHXYQ']['text']

510     Tacos were very bland. Queso tasted like it was from a can. Female bartender never once smiled. Won't be back.                                                                                                                                                                                                                                                                                                    
2721    This place is perfect if you are trying to impress someone on a date.   It has a great vibe and decor.  I would recommend it for drinks and appetizers.   The dishes here are good,  but take a different approach to flavor than the traditional Mexican cuisine.  For example the green enchiladas here don't come with a traditional tomatillo sauce.  Great for a date,  but don't expect traditional flavors.
4807    really liked the tacos - we had duck, lamb and a few others...and enjoyed the sangria. the building itself is just beautiful. love that it's been repurposed in such a pra

### Load DataSet

In [28]:
# Now lets get only 5k records for our training
sample_reviews = pd.read_csv("C:\\Users\\Aravind\\Documents\\Datasets\\Yelp review\\reviews_sample_5k.csv")

In [29]:
sample_reviews.shape

(4997, 10)

In [30]:
sample_reviews[sample_reviews['text'] == '']

,Unnamed: 0,review_id,user_id,business_id,stars,date,text,useful,funny,cool


In [31]:
pd.set_option('display.max_colwidth', 100)

sample_reviews.head()

,Unnamed: 0,review_id,user_id,business_id,stars,date,text,useful,funny,cool
0,1874445,oA_a09WGemmS7DsLa5ZWtQ,YQ98Qy__1H95mytfuPhZEw,hvhaAbjt_rm4v0e-cTK6GQ,1,2017-03-18,it's cute and very authentic looking. first impression was impressive. downhill from there. the ...,0,0,0
1,4395801,TEi95ne-JvUc36Rewo0Vdg,3cKi8BNKJT1OsWYMY4EivQ,6wpYVW5qEW5QmFgyX-xX9A,1,2013-09-17,"I came to Orangewood Animal Hospital for Dr Clark. With Dr Clark bringing on another doctor, my ...",5,4,2
2,2173300,DN-wFDPd64wj1O5ZY33bHQ,YUzVmcKvbk7NHr94SzIpMw,2QoIgLVJBreZvBDFu4pdcg,1,2017-04-22,A complete dump \nCrumbs on the bed and a hole in the bathroom door that had crumpled wood panel...,0,1,0
3,705316,7Feu2Q0Apl7N3bfF4CZauw,KuEqhi2AYfHVa4SXWmgWzw,CV6SRnXZliA9C3PEoIAHSg,1,2014-04-11,I am torn on how to review this place. I started using them 5 years ago when my male dogs were p...,8,1,1
4,3864276,vWIQFKNnYj541axoIRsr9g,C7Kj6xLZlzvXKzW-u5zLew,L24dRb0cIlEQQm0zVcrOwQ,1,2010-11-23,I had them come out to repair a couple of plumbing issues I was having that was not related to d...,0,0,0


In [32]:
sample_reviews.shape

(4997, 10)

In [33]:
# dropped all neutral ==3 reviews
sample_reviews.drop(sample_reviews[sample_reviews['stars'] == 3].index , inplace= True)

In [34]:
sample_reviews.shape

(4413, 10)

In [35]:
sample_reviews

,Unnamed: 0,review_id,user_id,business_id,stars,date,text,useful,funny,cool
0,1874445,oA_a09WGemmS7DsLa5ZWtQ,YQ98Qy__1H95mytfuPhZEw,hvhaAbjt_rm4v0e-cTK6GQ,1,2017-03-18,it's cute and very authentic looking. first impression was impressive. downhill from there. the ...,0,0,0
1,4395801,TEi95ne-JvUc36Rewo0Vdg,3cKi8BNKJT1OsWYMY4EivQ,6wpYVW5qEW5QmFgyX-xX9A,1,2013-09-17,"I came to Orangewood Animal Hospital for Dr Clark. With Dr Clark bringing on another doctor, my ...",5,4,2
2,2173300,DN-wFDPd64wj1O5ZY33bHQ,YUzVmcKvbk7NHr94SzIpMw,2QoIgLVJBreZvBDFu4pdcg,1,2017-04-22,A complete dump \nCrumbs on the bed and a hole in the bathroom door that had crumpled wood panel...,0,1,0
3,705316,7Feu2Q0Apl7N3bfF4CZauw,KuEqhi2AYfHVa4SXWmgWzw,CV6SRnXZliA9C3PEoIAHSg,1,2014-04-11,I am torn on how to review this place. I started using them 5 years ago when my male dogs were p...,8,1,1
4,3864276,vWIQFKNnYj541axoIRsr9g,C7Kj6xLZlzvXKzW-u5zLew,L24dRb0cIlEQQm0zVcrOwQ,1,2010-11-23,I had them come out to repair a couple of plumbing issues I was having that was not related to d...,0,0,0
5,1747650,gVNSlJFckpdIiik0Ugf0LQ,cek3m1xxsAu2zVgaSBnLWw,-o082vExIs0VVNSuZmiTQA,1,2015-04-26,"Terrible service. One star for decorations but took nearly 15 minutes to even order food, server...",1,0,0
6,2580458,jXQ3SqNNVp_-lNcOKs0pbg,Xkk67IIRCKhSeSn7b-cfEA,c8MUbiN3nX-YjK--WxO3cw,1,2017-12-10,Terrible customer service.. I tried calling ahead of time to ask questions and the live people I...,0,0,0
7,4438562,X72LjzwChiniVZpQyWHRVw,IiZv0qzwQYyZm9C_w_2RjA,a6SqJZTFMpSOR6Qb0STyFA,1,2011-10-31,"I am not a picky person. No really! I'm not, but I had such a horrible experience here I had t...",1,0,1
8,1687763,pFvpWDqob7M06CrVTl95NA,YCneW76-50jCwCV21Rm4sw,wGwib7KvPzs5_6aPCk_Fzw,1,2014-10-17,There's a golden light that scintillates from this place. It knows of hygiene and cleanliness. I...,1,0,0
9,4933015,LPp6CCIbC8DDhgznP9hMmw,Wr4uW7dG61pNx9EoHM1_EA,TT3LPP1k0kaw1nSsBS_SUA,1,2015-03-08,This is one of the most lowlife high-pressure sales establishment I experienced in many years.,4,5,0


In [36]:
# 1 --> Positive  0 --> Negative (1,2,4,5)
sample_reviews['stars']=sample_reviews.stars.apply(lambda x: 1 if x > 3 else 0)

In [37]:
sample_reviews


,Unnamed: 0,review_id,user_id,business_id,stars,date,text,useful,funny,cool
0,1874445,oA_a09WGemmS7DsLa5ZWtQ,YQ98Qy__1H95mytfuPhZEw,hvhaAbjt_rm4v0e-cTK6GQ,0,2017-03-18,it's cute and very authentic looking. first impression was impressive. downhill from there. the ...,0,0,0
1,4395801,TEi95ne-JvUc36Rewo0Vdg,3cKi8BNKJT1OsWYMY4EivQ,6wpYVW5qEW5QmFgyX-xX9A,0,2013-09-17,"I came to Orangewood Animal Hospital for Dr Clark. With Dr Clark bringing on another doctor, my ...",5,4,2
2,2173300,DN-wFDPd64wj1O5ZY33bHQ,YUzVmcKvbk7NHr94SzIpMw,2QoIgLVJBreZvBDFu4pdcg,0,2017-04-22,A complete dump \nCrumbs on the bed and a hole in the bathroom door that had crumpled wood panel...,0,1,0
3,705316,7Feu2Q0Apl7N3bfF4CZauw,KuEqhi2AYfHVa4SXWmgWzw,CV6SRnXZliA9C3PEoIAHSg,0,2014-04-11,I am torn on how to review this place. I started using them 5 years ago when my male dogs were p...,8,1,1
4,3864276,vWIQFKNnYj541axoIRsr9g,C7Kj6xLZlzvXKzW-u5zLew,L24dRb0cIlEQQm0zVcrOwQ,0,2010-11-23,I had them come out to repair a couple of plumbing issues I was having that was not related to d...,0,0,0
5,1747650,gVNSlJFckpdIiik0Ugf0LQ,cek3m1xxsAu2zVgaSBnLWw,-o082vExIs0VVNSuZmiTQA,0,2015-04-26,"Terrible service. One star for decorations but took nearly 15 minutes to even order food, server...",1,0,0
6,2580458,jXQ3SqNNVp_-lNcOKs0pbg,Xkk67IIRCKhSeSn7b-cfEA,c8MUbiN3nX-YjK--WxO3cw,0,2017-12-10,Terrible customer service.. I tried calling ahead of time to ask questions and the live people I...,0,0,0
7,4438562,X72LjzwChiniVZpQyWHRVw,IiZv0qzwQYyZm9C_w_2RjA,a6SqJZTFMpSOR6Qb0STyFA,0,2011-10-31,"I am not a picky person. No really! I'm not, but I had such a horrible experience here I had t...",1,0,1
8,1687763,pFvpWDqob7M06CrVTl95NA,YCneW76-50jCwCV21Rm4sw,wGwib7KvPzs5_6aPCk_Fzw,0,2014-10-17,There's a golden light that scintillates from this place. It knows of hygiene and cleanliness. I...,1,0,0
9,4933015,LPp6CCIbC8DDhgznP9hMmw,Wr4uW7dG61pNx9EoHM1_EA,TT3LPP1k0kaw1nSsBS_SUA,0,2015-03-08,This is one of the most lowlife high-pressure sales establishment I experienced in many years.,4,5,0


In [38]:
pd.set_option('display.max_colwidth',2000)
sample_reviews[sample_reviews['review_id']=='d_VhdHmBGU6uSiVOASVigQ']['text']

4996    Javier's is one of my favorite restaurants to hit while on the strip!  The ambience of Javier's is classy but at the same time live and full of energy.  This is a great place to go with big group to enjoy a dinner before a night out on the strip.  \n\nAlthough it is quite expensive for Mexican food, I have never been let down by any dish I have ordered here.  My go to order is La Tablita (para dos personas). Prepare your own tacos with your choice of $50 carne asada, carnitas or grilled natural chicken at your table. Served on a wooden platter with all the condiments: arroz, refritos, jalapenos, salsa fresca, guacamole,\nrajas, grilled green onions and your choice of warm tortillas.\n\nThis platter is enough to fill up 2 people easily and brings the cost to $25 dollars per person for the main entree.  \n\nI HIGHLY recommend ordering some Ceviche de Ahi and Guacamole as appetizers for the table.  The guacamole is made right in front of you and is absolutely fantastic.  Javier's 

In [39]:
len(set(nltk.corpus.stopwords.words('english')))

179

In [40]:
from sklearn.utils import shuffle

from nltk.stem import WordNetLemmatizer
from sklearn.linear_model import LogisticRegression
wordnet_lemmatizer = WordNetLemmatizer()


In [41]:
# from http://www.lextek.com/manuals/onix/stopwords1.html
custom_stopwords = set(w.rstrip() for w in open('C:\\Users\\Aravind\\Documents\\Datasets\\Yelp review\\stop_words.txt'))

In [42]:
custom_stopwords

{'a',
 'about',
 'above',
 'across',
 'after',
 'again',
 'against',
 'all',
 'almost',
 'alone',
 'along',
 'already',
 'also',
 'although',
 'always',
 'among',
 'an',
 'and',
 'another',
 'any',
 'anybody',
 'anyone',
 'anything',
 'anywhere',
 'are',
 'area',
 'areas',
 'around',
 'as',
 'ask',
 'asked',
 'asking',
 'asks',
 'at',
 'away',
 'b',
 'back',
 'backed',
 'backing',
 'backs',
 'be',
 'became',
 'because',
 'become',
 'becomes',
 'been',
 'before',
 'began',
 'behind',
 'being',
 'beings',
 'best',
 'better',
 'between',
 'big',
 'both',
 'but',
 'by',
 'c',
 'came',
 'can',
 'cannot',
 'case',
 'cases',
 'certain',
 'certainly',
 'clear',
 'clearly',
 'come',
 'could',
 'd',
 'did',
 'differ',
 'different',
 'differently',
 'do',
 'does',
 'done',
 'down',
 'downed',
 'downing',
 'downs',
 'during',
 'e',
 'each',
 'early',
 'either',
 'end',
 'ended',
 'ending',
 'ends',
 'enough',
 'even',
 'evenly',
 'ever',
 'every',
 'everybody',
 'everyone',
 'everything',
 'everyw

In [43]:
#nltk.corpus.stopwords.words('english')
stopwords.words('english')

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

###  Build the word count vector for each review

- ## tokenizers 

In [44]:
# we can also oversample the negative reviews
negative_reviews = sample_reviews[sample_reviews['stars'] == 0]['text']
positive_reviews = sample_reviews[sample_reviews['stars'] == 1]['text']


In [45]:
diff = len(positive_reviews) - len(negative_reviews)
idxs = np.random.choice(len(negative_reviews), size=diff)


In [46]:
diff

2193

In [47]:
# we can take 2193 negative reviews from our big dataset. I just oversampled to get the number from 5k dataset.
extra = negative_reviews[idxs]


In [48]:
negative_reviews = negative_reviews.append(extra)

In [49]:
print("no. of negative reviews",len(negative_reviews))
print("no. of positive reviews",len(positive_reviews))

no. of negative reviews 3303
no. of positive reviews 3303


In [50]:
# customer tokenizer

def yelp_tokenizer(s):
    s = s.lower() # downcase
    tokens = nltk.tokenize.word_tokenize(s) # split string into words (tokens)
    tokens = [t for t in tokens if len(t) > 2] # remove short words, they're probably not useful
    tokens = [wordnet_lemmatizer.lemmatize(t) for t in tokens] # put words into base form
    tokens = [t for t in tokens if t not in custom_stopwords] # remove stopwords
    return tokens


### Detour
- #### Tokenizer

In [51]:
sentence = "Their Carnita and Lamb tacos are divine - which is appropriate given the place is inside of a former church where the bar is wrapped around the former altar. "

In [52]:
sentence

'Their Carnita and Lamb tacos are divine - which is appropriate given the place is inside of a former church where the bar is wrapped around the former altar. '

In [53]:
# we can uise split() function to tokenize a sentense
# but, look at the full stop of the example at the end.
sentence.split()

['Their',
 'Carnita',
 'and',
 'Lamb',
 'tacos',
 'are',
 'divine',
 '-',
 'which',
 'is',
 'appropriate',
 'given',
 'the',
 'place',
 'is',
 'inside',
 'of',
 'a',
 'former',
 'church',
 'where',
 'the',
 'bar',
 'is',
 'wrapped',
 'around',
 'the',
 'former',
 'altar.']

In [54]:
tokenized_sentence = nltk.tokenize.word_tokenize(sentence)
tokenized_sentence

['Their',
 'Carnita',
 'and',
 'Lamb',
 'tacos',
 'are',
 'divine',
 '-',
 'which',
 'is',
 'appropriate',
 'given',
 'the',
 'place',
 'is',
 'inside',
 'of',
 'a',
 'former',
 'church',
 'where',
 'the',
 'bar',
 'is',
 'wrapped',
 'around',
 'the',
 'former',
 'altar',
 '.']

### Detour
- #### Stemming
- #### Lemmatizer

https://blog.bitext.com/what-is-the-difference-between-stemming-and-lemmatization/

In [55]:
stemmer = nltk.stem.PorterStemmer() 
print(stemmer.stem('appropriate'))
print(stemmer.stem('are'))
print(stemmer.stem('is'))
print(stemmer.stem('being'))

appropri
are
is
be


In [61]:
print(wordnet_lemmatizer.lemmatize('appropriate',pos = 'v'))
print(wordnet_lemmatizer.lemmatize('are',pos = 'v'))
print(wordnet_lemmatizer.lemmatize('is',pos = 'v'))
print(wordnet_lemmatizer.lemmatize('being',pos = 'v'))


# ,pos = 'v'

appropriate
be
be
be


In [62]:
[wordnet_lemmatizer.lemmatize(w,pos = 'v') for w in tokenized_sentence]

['Their',
 'Carnita',
 'and',
 'Lamb',
 'tacos',
 'be',
 'divine',
 '-',
 'which',
 'be',
 'appropriate',
 'give',
 'the',
 'place',
 'be',
 'inside',
 'of',
 'a',
 'former',
 'church',
 'where',
 'the',
 'bar',
 'be',
 'wrap',
 'around',
 'the',
 'former',
 'altar',
 '.']

In [63]:
[stemmer.stem(w) for w in tokenized_sentence]

['their',
 'carnita',
 'and',
 'lamb',
 'taco',
 'are',
 'divin',
 '-',
 'which',
 'is',
 'appropri',
 'given',
 'the',
 'place',
 'is',
 'insid',
 'of',
 'a',
 'former',
 'church',
 'where',
 'the',
 'bar',
 'is',
 'wrap',
 'around',
 'the',
 'former',
 'altar',
 '.']

In [64]:
# Do you see any difference here?
[nltk.stem.LancasterStemmer().stem(w) for w in tokenized_sentence]

['their',
 'carnit',
 'and',
 'lamb',
 'taco',
 'ar',
 'divin',
 '-',
 'which',
 'is',
 'appropry',
 'giv',
 'the',
 'plac',
 'is',
 'insid',
 'of',
 'a',
 'form',
 'church',
 'wher',
 'the',
 'bar',
 'is',
 'wrap',
 'around',
 'the',
 'form',
 'alt',
 '.']

In [65]:
# create a word-to-index map so that we can create our word-frequency vectors later
# let's also save the tokenized versions so we don't have to tokenize again later
word_index_map = {}
current_index = 0
positive_tokenized = []
negative_tokenized = []
orig_reviews = []
data = []




### Detour
#word_index_map

In [66]:
from ipywidgets import IntProgress
from IPython.display import display
import time


In [67]:
positive_tokenized

[]

In [68]:
max_count = len(positive_reviews)

f = IntProgress(min=0, max=max_count) # instantiate the bar
display(f) # display the bar

count = 0

for review in positive_reviews:

    orig_reviews.append(review)
    tokens = yelp_tokenizer(review)
    positive_tokenized.append(tokens)
    for token in tokens:
        if token not in word_index_map:
            word_index_map[token] = current_index
            current_index += 1
    count += 1
    f.value = count
    # signal to increment the progress bar
    f.description = '{:.2%}'.format(count/max_count)


IntProgress(value=0, max=3303)

In [69]:
positive_tokenized[1]

['food', 'wa', 'pretty', 'fantastic', 'definitely', 'creme', 'burlee']

In [70]:
# corresponding review text
orig_reviews[1]

'The food was pretty good, not fantastic, but I would definitely come back.  The creme burlee is a must.'

In [71]:
len(orig_reviews)

3303

In [72]:
max_count = len(negative_reviews)

f = IntProgress(min=0, max=max_count) # instantiate the bar
display(f) # display the bar

count = 0

for review in negative_reviews:

    orig_reviews.append(review)
    tokens = yelp_tokenizer(review)
    negative_tokenized.append(tokens)
    for token in tokens:
        if token not in word_index_map:
            word_index_map[token] = current_index
            current_index += 1
    count += 1    
    f.value = count # signal to increment the progress bar
    f.description = '{}/{}'.format(count,max_count)

IntProgress(value=0, max=3303)

In [49]:
negative_tokenized[2]

['complete',
 'dump',
 'crumb',
 'bed',
 'hole',
 'bathroom',
 'door',
 'crumpled',
 'wood',
 'panel',
 'paper',
 'hanging',
 'stay',
 'dump']

In [50]:
# correpsonding negative review.
# third from begining
orig_reviews[3303+2]

'A complete dump \nCrumbs on the bed and a hole in the bathroom door that had crumpled wood panel paper hanging from it\nStay away from this dump'

In [51]:
# our universe of words (ie. every words here is one feature and feature vectors will be built using ths as reference))
word_index_map

{'giving': 0,
 'star': 1,
 'based': 2,
 'previous': 3,
 'experience': 4,
 'recent': 5,
 'fresh': 6,
 'start': 7,
 'literally': 8,
 'yummy': 9,
 'day': 10,
 'service': 11,
 'nor': 12,
 'presentation': 13,
 'food': 14,
 'taste': 15,
 'typically': 16,
 'alternative': 17,
 'truly': 18,
 'usually': 19,
 'packed': 20,
 'breakfast': 21,
 'time': 22,
 '...': 23,
 'traditional': 24,
 'egg': 25,
 'meat': 26,
 'toast': 27,
 'potato': 28,
 'visit': 29,
 '11:30am': 30,
 'sunday': 31,
 'wa': 32,
 'bad': 33,
 "n't": 34,
 'busy': 35,
 'odd': 36,
 'sibling': 37,
 'wait': 38,
 'top': 39,
 'benedict': 40,
 'brother': 41,
 'corned': 42,
 'beef': 43,
 'sister': 44,
 'portabella': 45,
 'mushroom': 46,
 'crab': 47,
 'cake': 48,
 'disappointing': 49,
 'looked': 50,
 'sort': 51,
 'deep': 52,
 'fried': 53,
 'hush': 54,
 'puppy': 55,
 'mushy': 56,
 'consistency': 57,
 'tasted': 58,
 'grease': 59,
 'proud': 60,
 'serve': 61,
 'bleggch': 62,
 'hollandaise': 63,
 'sauce': 64,
 'eat': 65,
 'spoonful': 66,
 'pretty':

In [73]:
positive_tokenized[2]

['friend',
 'eat',
 'wa',
 'close',
 'hotel',
 'mexican',
 'awhile',
 'food',
 'wa',
 'waitress',
 'feel',
 'family',
 'wa',
 'helpful',
 'suggestion',
 'care',
 'friend',
 'special',
 'request',
 'wa',
 "n't",
 'menu',
 'overall',
 'dining',
 'experience',
 'average',
 'food',
 'olive',
 'garden',
 'stuff',
 'waitress',
 'left',
 'happy',
 'laughing']

In [86]:
# Function to create feature vectors for a given token set (==review)

def tokens_to_vector(tokens, label):
    x = np.zeros(len(word_index_map) + 1) # last element is for the label
    
    for t in tokens:
        i = word_index_map[t]
        x[i] += 1
#     print(x.sum())
    if x.sum() != 0:
        x = x / x.sum() # normalize it before setting label
    x[-1] = label
    return x

In [77]:
#Total reviews 
N = len(positive_tokenized) + len(negative_tokenized)
print(N)

6606


In [78]:
orig_reviews[1]

'The food was pretty good, not fantastic, but I would definitely come back.  The creme burlee is a must.'

In [79]:
positive_tokenized[1]

['food', 'wa', 'pretty', 'fantastic', 'definitely', 'creme', 'burlee']

In [87]:
numpy.set_printoptions(threshold=numpy.nan)
feature_vector = tokens_to_vector(positive_tokenized[5],1)
print(feature_vector.shape)
print(feature_vector)
# observe that its a sparse vector with only a handfull of featrues having values not 0
# try for 5th positive_tokenized[4] positive review. Do you see more non zero values?
# Observe that values are (frequency) normalized

(19639,)
[0.         0.         0.         0.         0.02857143 0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.02857143 0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.11428571 0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.02857143 0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.02857143 0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.

In [88]:
np.seterr(divide='ignore', invalid='ignore')

i = 0
data = np.zeros((N, len(word_index_map) + 1))
print(data.shape)

for tokens in positive_tokenized:
    xy = tokens_to_vector(tokens, 1)
    data[i,:] = xy
    i += 1
print(data.shape)
for tokens in negative_tokenized:
    xy = tokens_to_vector(tokens, 0)
    data[i,:] = xy
    i += 1
print(data.shape)

(6606, 19639)
(6606, 19639)
(6606, 19639)


In [89]:
len(orig_reviews)

6606

In [98]:
# shuffle the data and create train/test splits
# try it multiple times!
for i in range(0,10):
    orig_reviews, data = shuffle(orig_reviews, data)

X = data[:,:-1] # H --> Features
Y = data[:,-1]
print(Y[0])

1.0


In [99]:
# last 500 rows will be test
Xtrain = X[:-500,]
Ytrain = Y[:-500,]
Xtest = X[-500:,]
Ytest = Y[-500:,]


In [ ]:
Xtrain[0]


In [123]:

fg =[[1,2,3],[3,4,5],[6,7,8]]

xc = fg[:-1,]
ec = fg[-1:]
print(xc)
print(ec)

TypeError: list indices must be integers or slices, not tuple

In [128]:
Xtrain[1:3]


array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.  

In [129]:
Ytrain[1:3]
# categorical (0,1)

array([1., 0.])

### Detour
#### LogisticRegression

In [130]:

model = LogisticRegression()
model.fit(Xtrain, Ytrain)
print("Train accuracy:", model.score(Xtrain, Ytrain))
print("Test accuracy:", model.score(Xtest, Ytest))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Train accuracy: 0.8381919423517852
Test accuracy: 0.838


In [131]:
word_index_map['literally']

8

In [66]:
# let's look at the weights for each word
# try it with different threshold values!
# check the words rude/sad, sorry/awful
# check the words super, absolutely, favorite, recommend, excellent .. get any idea?
threshold = 0.5
for word, index in word_index_map.items():
    weight = model.coef_[0][index]
    if weight > threshold or weight < -threshold:
        print(word, weight)


star -0.576504702892306
experience 0.679506848024407
fresh 2.4388742058069597
literally -1.0960664038419419
yummy 0.9017627298973478
service 0.5586203757932712
breakfast 0.9606784899868364
... -2.5097113021124975
meat -0.6172385499110469
visit 0.5601085736750908
wa -3.1545671964150044
bad -2.789338398712465
n't -6.7001290783063245
disappointing -0.9294164596661281
looked -0.774543803917583
tasted -0.7908667497242169
sauce 0.5479583271047366
pretty 1.1461017813664314
fantastic 1.8173040493373331
definitely 4.258815313637851
close -0.5484932054631997
waitress -1.219563435726717
feel 0.6814656663430321
family 0.6362740801376652
helpful 1.866565460843062
special 0.8852520534031935
left -1.3506978230399815
happy 1.4821361952193084
fun 2.0250794458664134
little 1.7832008934880546
kid 0.6406127626240727
people -2.5630540326449758
friendly 4.971209946710938
dinner 0.8844353092467072
love 5.527742172735191
try 1.8852769433095828
portion 1.29233975050004
perfectly 0.7232801790206789
style 0.6444

In [67]:
len(word_index_map)

19649

In [68]:
# check misclassified examples
preds = model.predict(X)
P = model.predict_proba(X)[:,1] # p(y = 1 | x)


In [69]:
# since there are many, just print the "most" wrong samples
minP_whenYis1 = 1
maxP_whenYis0 = 0
wrong_positive_review = None
wrong_negative_review = None
wrong_positive_prediction = None
wrong_negative_prediction = None
for i in range(N):
    p = P[i]
    y = Y[i]
    if y == 1 and p < 0.5:
        if p < minP_whenYis1:
            wrong_positive_review = orig_reviews[i]
            wrong_positive_prediction = preds[i]
            minP_whenYis1 = p
    elif y == 0 and p > 0.5:
        if p > maxP_whenYis0:
            wrong_negative_review = orig_reviews[i]
            wrong_negative_prediction = preds[i]
            maxP_whenYis0 = p

In [70]:
## Need to work on this

## The review which is wrongly classified as NEGATIVE
print("Wrong most positive review (prob = %s, pred = %s):" % (minP_whenYis1, wrong_positive_prediction))
print(wrong_positive_review)

## The review which is wrongly classified as POSITIVE
print("Most wrong negative review (prob = %s, pred = %s):" % (maxP_whenYis0, wrong_negative_prediction))
print(wrong_negative_review)


Wrong most positive review (prob = 0.20883041428778054, pred = 0.0):
I thought it was a great show. It was almost too loud and at some points there was so much going on it was hard to stay focused. All in all it kept me entertained.
Most wrong negative review (prob = 0.7281261888570476, pred = 1.0):
The $$ is incorrect. This place is definitely $$$. I loved the atmosphere, decor, and music. However disappointed with the menu and high prices.


In [71]:
confusion_matrix(Y,preds)

array([[2880,  423],
       [ 620, 2683]])

In [72]:
print("index   Y_real  Y_pred     Probability     review")
print("-" *100)
for i in range(20):
    print("{}\t{} \t {}\t {:.2%} \t {}..".format(i,Y[i], preds[i],P[i],orig_reviews[i][:70]))
    

#Y_real Y_pred  
#  1       1              True Positve
#  0       0              True Negative
#  0       1              False Positive
#  1       0              False Negative

index   Y_real  Y_pred     Probability     review
----------------------------------------------------------------------------------------------------
0	1.0 	 1.0	 57.65% 	 Great upscale restaurant in Cave Creek.  They specialize in exotic mea..
1	1.0 	 1.0	 51.57% 	 We ate dinner there last night. THE FAJITAS were great and service was..
2	0.0 	 0.0	 44.18% 	 I purchased a 2017 Toyota Camry from Dave Wilson's Toyota October 29th..
3	1.0 	 1.0	 67.37% 	 Casual place with a spacious and very clean dinning floor. 

Chicken k..
4	1.0 	 1.0	 75.35% 	 Whenever we order delivery it's from Banjara. Delicious Indian, at a g..
5	1.0 	 0.0	 47.27% 	 We had lunch here on our first day in Vegas.  The food was excellent, ..
6	0.0 	 0.0	 22.11% 	 Horrible Service. The plumber left the problem worse that what it was ..
7	1.0 	 1.0	 71.60% 	 Outstanding, pizza by the slice, awesome, lasagne...awesome, sub sandw..
8	0.0 	 0.0	 43.66% 	 Always slow service. They don't always have what you initially want

In [73]:
# second review is false negative.  marginal case with prob=49.44%.
orig_reviews[1]

'We ate dinner there last night. THE FAJITAS were great and service was also good. It is worth the drive from Henderson.'

In [74]:
model.classes_

array([0., 1.])